### Data
The code below reads in Geotiff files from NASA Socioeconomic Data and Applications Center (SEDAC)
Documentation for the U.S. Social Vulnerability Index Grids, v1 (2000, 2010, 2014, 2016, 2018).

Using the gdal from the osgeo package, latitude and longitude values are generated for each pixel in the raster.
The raster values are then read into a Pandas DataFrame with the latitude and longitude values as columns.
The resulting DataFrame is then merged with the other years to create a single DataFrame with all years and economic values whcih include: Socioeconomic Ranking, Household Composition & Disability, Minority Status & Language, Housing Type & Transportation, and overall Social Vulnerability Index Ranking.

In [1]:
#base dependencies
import pandas as pd
import numpy as np

#GIS packages
from osgeo import gdal
from geopy.geocoders import Nominatim
import geopandas as gpd
import georasters as gr

#utility packages
import os
import requests
from bs4 import BeautifulSoup
import re

#plots
import plotly.express as px

### Raw Data

In [2]:
# Raw data as geoTifs hosted on GitHub Repo
tif_urls = ["https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2014_tract_household_nad83.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2014_tract_housing_nad83.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2014_tract_minority_nad83.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2014_tract_overall_nad83.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2014_tract_socioeconomic_nad83.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2016_tract_household_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2016_tract_housing_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2016_tract_minority_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2016_tract_overall_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2016_tract_socioeconomic_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2018_tract_household_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2018_tract_housing_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2018_tract_minority_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2018_tract_overall_nad83_nopop.tif",
            "https://github.com/DanDryer/Team-Project-Practicum-6748/raw/main/SEDAC_SVI_data/svi_2018_tract_socioeconomic_nad83_nopop.tif"
            ]



### Data Cleaning

In [3]:
def geotif_to_pandas(path, value_name):
    """
    This function converts a GeoTIFF file to a GeoDataFrame and adds latitude and longitude values for each pixel
    :param path: The path to the GeoTIFF file
    :param value_name: The name of the column for raster values in the GeoDataFrame
    :return: A GeoDataFrame with latitude, longitude, and raster values
    """
    # Read the GeoTIFF file 
    geotif = gr.from_file(path)
    
    #converts to pandas df
    geodf = geotif.to_pandas()

    # Extract the latitude, longitude, and raster values
    df = geodf.loc[:,[True, # Raster values
                      'x', # Latitude
                      'y']]# Longitude
    
    # Rename columns
    column_names = {True: value_name, 'x': 'Latitude', 'y' : 'Longitude'} 
    df = df.rename(columns=column_names)
    
    return df

def add_county_zip(df):
    
    """
    This function uses the latitude and longitude data and a Python client called Geopy to find and add the county information to the dataset.
    :param df: The dataframe that the county information will be added to
    :return: A Pandas DataFrame with latitude, longitude, raster values, and county
    """

    df['coords'] = df['Latitude'].map(str) + ',' + df['Longitude'].map(str)
    
    county = []
    for i in df['coords']:
        locator = Nominatim(user_agent='myGeocoder')
        location = locator.reverse(i)
        value = (location.raw['address']).get('county')
        county.append(value)

    df['County'] = county
    
    zip = []
    for i in df['coords']:
        locator = Nominatim(user_agent='myGeocoder')
        location = locator.reverse(i)
        value = (location.raw['address']).get('postcode')
        zip.append(value)

    return df

In [ ]:
### Warning this cell is computationally expensive

# List of dfs for each processed geotiff file
dfs = []

# Iterate through tif_urls and process each file
for tif_url in tif_urls:
    # Extract the year and value name from the URL
    filename = os.path.basename(tif_url)
    match = re.search(r"svi_(\d+)_tract_([a-zA-Z]+)", filename)
    if match:
        year = match.group(1)
        value_name = year + "_" + match.group(2)

        # Call geotif_to_pandas function with the appropriate arguments
        df = geotif_to_pandas(tif_url, value_name)
        
        # Append the df to list
        dfs.append(df)
    else:
        pass
    
# Perform inner join on 'Latitude' and 'Longitude'
df_merged = dfs[0]  # Start with the first DataFrame

for df in dfs[1:]:
    df_merged = pd.merge(df_merged, df, on=['Latitude', 'Longitude'], how='inner')

### could not run this, I kept getting errors related to the Nominatum function ###
#df_merged = add_county_zip(df_merged)

df_svi = df_merged

In [6]:
df_merged.describe()

,2014_household,Latitude,Longitude,2014_housing,2014_minority,2014_overall,2014_socioeconomic,2016_household,2016_housing,2016_minority,2016_overall,2016_socioeconomic,2018_household,2018_housing,2018_minority,2018_overall,2018_socioeconomic
count,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06,9.153038e+06
mean,5.890963e-01,-1.001339e+02,4.104003e+01,5.427965e-01,3.270933e-01,4.974725e-01,5.044819e-01,5.907746e-01,5.333630e-01,3.183558e-01,4.960319e-01,5.090534e-01,6.012385e-01,5.322033e-01,3.154114e-01,5.002789e-01,5.172087e-01
std,2.393126e-01,1.856516e+01,8.028895e+00,2.481619e-01,2.510189e-01,2.445421e-01,2.375708e-01,2.389508e-01,2.572207e-01,2.479444e-01,2.482178e-01,2.398771e-01,2.362909e-01,2.590478e-01,2.444966e-01,2.494314e-01,2.406032e-01
min,0.000000e+00,-1.781750e+02,1.892500e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.028000e-01,-1.090667e+02,3.555000e+01,3.428000e-01,1.162000e-01,3.002000e-01,3.135000e-01,4.169000e-01,3.193000e-01,1.116250e-01,2.923000e-01,3.123000e-01,4.295000e-01,3.150000e-01,1.130000e-01,2.946000e-01,3.161000e-01
50%,6.123000e-01,-9.685833e+01,4.020833e+01,5.459000e-01,2.554000e-01,4.927000e-01,5.053000e-01,6.184000e-01,5.331000e-01,2.503000e-01,4.918000e-01,5.082000e-01,6.290000e-01,5.362000e-01,2.546000e-01,4.982000e-01,5.144000e-01
75%,7.853000e-01,-8.725000e+01,4.465833e+01,7.546000e-01,5.102000e-01,6.856000e-01,6.956000e-01,7.808000e-01,7.564000e-01,4.953000e-01,6.887000e-01,7.044000e-01,7.886000e-01,7.546000e-01,4.748000e-01,6.947000e-01,7.105000e-01
max,1.000000e+00,-6.696667e+01,7.136667e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999000e-01,1.000000e+00,9.999000e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


### Data Visualization

In [ ]:
# Visualization is displayed inline
%matplotlib inline

# Set your Mapbox access token
mapbox_access_token = 'pk.eyJ1IjoianJhc2hpZCIsImEiOiJjbGlsMXhjZ3EwNzF3M2VwZWE1NDZtMjB4In0.CesHDGB6S6qDVDtTTeQxvw'

# Define the base figure using the first variable
fig = px.scatter_mapbox(df_svi, lat='Latitude', lon='Longitude',
                        hover_data=[df_svi.columns[2]], zoom=10)

# Add toggling options for each variable
for column in merged_df.columns[3:]:
    fig.add_trace(px.scatter_mapbox(df_svi,
                                    lat='Latitude',
                                    lon='Longitude',
                                    hover_data=[column]).data[0])

# Update the layout to include map style and title
fig.update_layout(mapbox_style="open-street-map", mapbox_accesstoken=mapbox_access_token)
fig.update_layout(title="Interactive Point Map")

# Show the interactive plot inline in the Jupyter Notebook
fig.show(renderer='notebook')


### Creating Additional Values